In [3]:
import copy
from collections import defaultdict
from itertools import product

In [4]:
def init(fname):
    state = {1:['ELE'], 2:[], 3:[], 4:[]}
    with open(fname, 'r') as inf:
        floor = 0
        for line in inf:
            floor += 1
            tokens = line.split()
            for i, token in enumerate(tokens):
                if '-compatible' in token:
                    typ = token.split('-')[0][0:2].upper()
                    state[floor].append(typ + 'M')
                if token.startswith('generator'):
                    state[floor].append(tokens[i-1][0:2].upper() + 'G')
    return state

In [10]:
class Success(Exception):
    pass

def find_gen(state, gen):
    for floor, objs in state.items():
        if gen in objs:
            return floor

def effective_state(state):
    res = {1:[], 2:[], 3:[], 4:[]}
                
    for floor, objs in state.items():
        for o in objs:
            if o == 'ELE':
                res[floor].append(o)
            if o[2] == 'M':
                res[floor].append(f'M{floor}')
                gen = f'{o[0:2]}G'
                res[find_gen(state, gen)].append(f'G{floor}')

    return res
           
def valid(state):
    for floor, objs in state.items():
        all_objs = ''.join(objs)
        if 'G' not in all_objs:
            continue  # No generator on this floor
        for o in objs:
            if o[2] == 'M' and o[0:2]+'G' not in all_objs:
                return False
    return True

def success(state):
    for floor, objs in state.items():
        if floor != 4 and len(objs) != 0:
            return False
            
    return True

def get_cur_floor(state):
    for floor, objs in state.items():
        if 'ELE' in objs:
            return floor, [o for o in objs if o != 'ELE']
    
def advance(states, known_states):
    newstates = []
    for state in states:
        floor, objs = get_cur_floor(state)
        for direct in [1, -1]:
            # Assume, that we never have to move to a lower level, if
            # there is nothing on any lower level
            if direct == -1 and floor > 1:
                nothing_below = True
                for lower in range(floor-1, 0, -1):
                    if len(state[lower]) != 0:
                        nothing_below = False
                if nothing_below:
                    continue
            if floor + direct > 4 or floor + direct < 1:
                continue
            for comb in product(objs, repeat=2):
                newstate = copy.deepcopy(state)
                newstate[floor].remove('ELE')
                newstate[floor+direct].append('ELE')
                newstate[floor].remove(comb[0])
                newstate[floor+direct].append(comb[0])
                if comb[0] != comb[1]:
                    newstate[floor].remove(comb[1])
                    newstate[floor+direct].append(comb[1])
                if not valid(newstate):
                    continue
                if success(newstate):
                    raise Success('Success!')
                eff_state = effective_state(newstate)
                if newstate not in newstates and eff_state not in known_states:
                    newstates.append(newstate)
                    known_states.append(eff_state)
    return newstates, known_states

In [11]:
def get_number_of_moves(infile):
    start = init(infile)

    states = [start]
    effstates = [effective_state(start)]

    for step in range(1, 100):
        try:
            # print(step, len(states))
            states, effstates = advance(states, effstates)
        except Success:
            return step

In [13]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

n_moves = get_number_of_moves('input11a.txt')

print(f'Number of moves: {n_moves}')

assert n_moves == 11

print('\nPuzzle case\n')

n_moves = get_number_of_moves('input11.txt')

print(f'Number of moves: {n_moves}')

assert n_moves == 33

print('\n*****\nPuzzle2\n*****\n')

print('Puzzle case\n')

n_moves = get_number_of_moves('input11_2.txt')

print(f'Number of moves: {n_moves}')

assert n_moves == 57


*****
Puzzle1
*****

Test case

Number of moves: 11

Puzzle case

Number of moves: 33

*****
Puzzle2
*****


Puzzle case

Number of moves: 57
